<a href="https://colab.research.google.com/github/Wasifnasim/projects-/blob/main/Pothole_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install ultralytics==8.* opencv-python roboflow fastapi uvicorn


import torch, ultralytics, cv2, os, sys
print('Torch CUDA available:', torch.cuda.is_available())
print('Ultralytics version:', ultralytics.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 117.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch CUDA available: True
Ultralytics version: 8.3.184


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="AAplNv63SCyyYAn4RiGg")
project = rf.workspace("pothole-pgqso").project("yolo-v8-project-jkmun")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to yolo-v8-project-1 in yolov8:: 100%|██████████| 412/412 [00:00<00:00, 5403.14it/s]


In [ ]:
!unzip '/content/yolo v8 project.v1-yolo-v8-.yolov8-obb.zip' -d datasets/
!ls datasets


Archive:  /content/yolo v8 project.v1-yolo-v8-.yolov8-obb.zip
  inflating: datasets/README.dataset.txt  
  inflating: datasets/README.roboflow.txt  
  inflating: datasets/data.yaml      
   creating: datasets/test/
   creating: datasets/test/images/
 extracting: datasets/test/images/pic-102-_jpg.rf.f0ab5d04fa24d6233af06ff27a24c30a.jpg  
 extracting: datasets/test/images/pic-108-_jpg.rf.24a29aaf913119696a254a69d17f388b.jpg  
 extracting: datasets/test/images/pic-11-_jpg.rf.76dfc8f59a4496f1f66358c99c9fb9e2.jpg  
 extracting: datasets/test/images/pic-111-_jpg.rf.8b2f3d2a1c29dc6f16711892d2118d8e.jpg  
 extracting: datasets/test/images/pic-126-_jpg.rf.361504a61eac88c29bd3e05114326e4d.jpg  
 extracting: datasets/test/images/pic-128-_jpg.rf.951a6fbc9af798d539cc528fe238aa47.jpg  
 extracting: datasets/test/images/pic-13-_jpg.rf.a4d891f9408d5b36aaa7101c796cd6ad.jpg  
 extracting: datasets/test/images/pic-135-_jpg.rf.571aa4d4754cea8d34283c612f571799.jpg  
 extracting: datasets/test/images/pic-14

In [ ]:
train:'/content/datasets/train/images'
val: '/content/datasets/valid'
test: '/content/datasets/test'
nc: 1
names: ['pothole']


In [ ]:
import os, glob, yaml

# If Step 2 already defined these, keep them. Otherwise, MANUALLY set like:
# DATA_ROOT = '/content/datasets/<your-dataset-folder>'      # e.g. '/content/datasets/Potholes-3'
# DATA_YAML = f'{DATA_ROOT}/data.yaml'

assert os.path.exists('/content/yolo-v8-project-1/data.yaml'), f"data.yaml not found at {DATA_YAML}"
print("----- data.yaml -----")
print(open('/content/yolo-v8-project-1/data.yaml').read())

splits = ['train', 'val', 'test']
for s in splits:
    img_dir = os.path.join('DATA_ROOT', s, 'images')
    lab_dir = os.path.join('DATA_ROOT', s, 'labels')
    imgs = glob.glob(os.path.join(img_dir, '*'))
    labs = glob.glob(os.path.join(lab_dir, '*.txt'))
    print(f"{s:>5}: {len(imgs)} images | {len(labs)} labels -> {img_dir}")


----- data.yaml -----
names:
- Pothole_Segmentation_YOLOv8 - v1 2023-10-20 10-09pm
nc: 1
roboflow:
  license: CC BY 4.0
  project: yolo-v8-project-jkmun
  url: https://universe.roboflow.com/pothole-pgqso/yolo-v8-project-jkmun/dataset/1
  version: 1
  workspace: pothole-pgqso
test: ../test/images
train: ../train/images
val: ../valid/images

train: 0 images | 0 labels -> DATA_ROOT/train/images
  val: 0 images | 0 labels -> DATA_ROOT/val/images
 test: 0 images | 0 labels -> DATA_ROOT/test/images


In [ ]:
from ultralytics import YOLO


MODEL_NAME = 'pothole_y8s'
IMG = 896
EPOCHS = 100
BATCH = 16


model = YOLO('yolov8s.pt') # pretrained weights
results = model.train(
data=('/content/yolo-v8-project-1/data.yaml'),
imgsz=IMG,
epochs=EPOCHS,
batch=BATCH,
name=MODEL_NAME,
patience=25, # early stopping
device=0 if torch.cuda.is_available() else 'cpu',
)


print('Training finished. Best weights:')
!ls -l /content/runs/detect/$MODEL_NAME/weights

Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo-v8-project-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pothole_y8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=25, perspective=0.0, plots=True, pose=12.0,

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

Model summary: 129 layers, 11,135,987 parameters, 11,135,971 gradients, 28.6 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 878.4±508.9 MB/s, size: 73.3 KB)


train: Scanning /content/yolo-v8-project-1/train/labels... 144 images, 0 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<00:00, 1914.69it/s]

...

    100/100      6.66G     0.5273     0.3825     0.9689         66        896: 100%|██████████| 9/9 [00:04<00:00,  1.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.77it/s]

                   all         35         94      0.925      0.914       0.94       0.71



100 epochs completed in 0.181 hours.
Optimizer stripped from runs/detect/pothole_y8s/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/pothole_y8s/weights/best.pt, 22.6MB

Validating runs/detect/pothole_y8s/weights/best.pt...
Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.45it/s]


                   all         35         94      0.922      0.915      0.939       0.71
Speed: 0.7ms preprocess, 8.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/pothole_y8s
Training finished. Best weights:
total 44096
-rw-r--r-- 1 root root 22574506 Aug 24 06:31 best.pt
-rw-r--r-- 1 root root 22574506 Aug 24 06:31 last.pt
